In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sns
import geopandas as gpd

import cartopy.crs as ccrs
import scipy.stats as stats

from matplotlib import patches
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec

import sys
sys.path.append(os.path.abspath(".."))
from function import ART_downscale as ART_down

import warnings
warnings.filterwarnings('ignore')

## Export Number of wet days, Weibull Parameters and Quantiles
### Quantiles_OBS info about the stations and quantiles for all period
### WEIBULL_OBS Wet days, Weibull parameters for each year

In [2]:
dir_base = os.path.join('/','media','arturo','Arturo','Data','Italy')

In [3]:
OBS_base = os.path.join(dir_base,'stations','data')
OBS_INFO_dir = os.path.join(os.path.join(OBS_base),'Veneto','BACKUP_RAW','VENETO_INFO_1dy.csv')

if os.path.exists(OBS_INFO_dir):
    INFO_pd = pd.read_csv(OBS_INFO_dir, sep=',')
else:
    raise SystemExit(f"File not found: {OBS_INFO_dir}")

In [4]:
names, lats, lons, yys, yye, nys, elev, Ns, Cs, Ws = [], [], [], [], [], [], [], [], [], []

for nn in range(len(INFO_pd)):
    RAIN_pd = pd.read_csv(os.path.join(OBS_base,'Veneto','BACKUP_RAW','CLEAR_1dy', f'{INFO_pd['File_Name'].values[nn]}.csv'), sep=',')
    RAIN_pd['TIME'] = pd.to_datetime(RAIN_pd['TIME'].values)
    ss, ee = RAIN_pd['TIME'][0].year, RAIN_pd['TIME'][len(RAIN_pd['TIME'])-1].year
    ny = (ee-ss)+1

    names.append(INFO_pd['File_Name'].values[nn])
    lats.append(INFO_pd['Lat'].values[nn])
    lons.append(INFO_pd['Lon'].values[nn])
    yys.append(ss)
    yye.append(ee)
    nys.append(ny)
    elev.append(INFO_pd['Elv'].values[nn])

Quantiles_OBS = pd.DataFrame({'File_Name':names, 
                    'Lat':lats, 'Lon':lons, 
                    'YYS':yys, 'YYE':yye, 
                    'Elv':elev,'NY':nys})

In [5]:
names, lats, lons, Q10, Q20, Q50, Q100, Q200 = [], [], [], [], [], [], [], []

for nn in range(len(Quantiles_OBS)):
    filename = f'{Quantiles_OBS['File_Name'].values[nn]}.csv'
    RAIN_pd = pd.read_csv(os.path.join(OBS_base,'Veneto','BACKUP_RAW','CLEAR_1dy', filename), sep=',')
    RAIN_pd['TIME'] = pd.to_datetime(RAIN_pd['TIME'].values)
    ss, ee = RAIN_pd['TIME'][0].year, RAIN_pd['TIME'][len(RAIN_pd['TIME'])-1].year
    ny = (ee-ss)+1
    year_list = np.arange(ss,ee+1)

    if ny != len(year_list):
        raise SystemExit(f"ERROR years with differents lenghts")

    RAIN_xr = xr.DataArray(
                    RAIN_pd['PRE'].values,
                    coords={'time':RAIN_pd['TIME'].values}, 
                    dims=('time'))

    NCW = ART_down.fit_yearly_weibull_update(RAIN_xr,1)
    N = NCW[:,0]
    C = NCW[:,1]
    W = NCW[:,2]
    Y = NCW[:,3].astype('int')

    Tr = [10, 20, 50, 100, 200]
    Fi = 1 - 1/np.array(Tr)     
    x0 = np.nanmean(C)
    TMP_Q, flag = ART_down.mev_quant(Fi, 10, N, C, W, thresh=1)
    TMP_Q2 = np.where(flag, TMP_Q, np.nan)
    Q10.append(np.round(TMP_Q2[0],4))
    Q20.append(np.round(TMP_Q2[1],4))
    Q50.append(np.round(TMP_Q2[2],4))
    Q100.append(np.round(TMP_Q2[3],4))
    Q200.append(np.round(TMP_Q2[3],4))

Quantiles_OBS['Q10'] = Q10
Quantiles_OBS['Q20'] = Q20
Quantiles_OBS['Q50'] = Q50
Quantiles_OBS['Q100'] = Q100
Quantiles_OBS['Q200'] = Q200

Quantiles_OBS = Quantiles_OBS[~np.isnan(Quantiles_OBS['Q50'])]
Quantiles_OBS = Quantiles_OBS.reset_index(drop=True)

ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve does not work - change x0
ERROR - fsolve d

In [6]:
Quantiles_OBS

,File_Name,Lat,Lon,YYS,YYE,Elv,NY,Q10,Q20,Q50,Q100,Q200
0,002_FG_BS,45.895600,12.815000,1995,2022,21,28,132.4365,154.8719,186.0192,211.0015,211.0015
1,003_BL_Ar,46.499984,11.876073,1984,2023,1642,40,92.7934,106.9952,126.0768,140.9461,140.9461
2,003_FG_BS,45.918000,12.545000,1995,2022,22,28,158.0486,291.6953,636.4766,1007.8025,1007.8025
3,004-264_NP_Be,46.164679,12.244708,1993,2023,379,31,120.6487,137.5578,160.0625,177.4375,177.4375
4,004_NP_Be,46.133632,12.195395,1993,2006,392,14,111.3868,126.9152,147.6461,163.6962,163.6962
...,...,...,...,...,...,...,...,...,...,...,...,...
222,345_NP_Ga,45.880221,11.552810,2002,2023,1176,22,149.5789,176.3166,213.2328,242.7867,242.7867
223,348_NP_Ro,46.190733,12.407890,1992,2023,1160,32,154.9915,178.7045,210.8762,236.1598,236.1598
224,349_NP_Tr,45.663472,12.237917,1987,2015,13,29,101.5571,118.4892,142.2526,161.5775,161.5775
225,350_NP_Va,45.862455,11.659233,1984,2023,148,40,193.8861,228.6844,276.3371,314.0748,314.0748


In [7]:
dirout = os.path.join(OBS_base,'Quantiles','Veneto_Q.csv')
Quantiles_OBS.to_csv(dirout,header=True,index=False)